In [1]:
import umap
import hdbscan
import numpy as np
import pandas as pd
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn.cluster as cluster
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_openml
# Dimension reduction and clustering libraries
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score
sns.set(context= 'notebook', style= 'white', rc={'figure.figsize':(14,10)})
%matplotlib inline

In [2]:
%cd Z:\PPMI_Data\Excels\Raw_And_More
df = pd.read_csv ("beta_post_Funnorm_PPMI_EPICn524final030618.csv")
df = df.round(decimals = 4)
df.head(2)

Z:\PPMI_Data\Excels\Raw_And_More


,200973410159_R03C01,200991620021_R01C01,200991620021_R03C01,200989060105_R03C01,200989060135_R07C01,200989060162_R04C01,200973410167_R04C01,200973410143_R01C01,200973410143_R06C01,200973410167_R05C01,...,200989060089_R02C01,200989060089_R01C01,200989060089_R04C01,200973410143_R08C01,200973410143_R07C01,200989060089_R03C01,200989060150_R04C01,200989060150_R05C01,200989060150_R06C01,200991620049_R08C01
cg14817997,0.8340,0.8492,0.7730,0.8853,0.7725,0.7564,0.7198,0.7759,0.7689,0.7594,...,0.7882,0.8389,0.8453,0.7704,0.8529,0.7838,0.8069,0.8757,0.6613,0.7379
cg26928153,0.9456,0.9485,0.9622,0.9440,0.9263,0.9561,0.9130,0.9560,0.9571,0.8976,...,0.9512,0.9590,0.9571,0.9547,0.9478,0.9581,0.9480,0.9424,0.9550,0.9463


In [3]:
df.index.name ='CpG'
df.reset_index(inplace=True)
print(df.shape)
df.head(2)

(864067, 525)


,CpG,200973410159_R03C01,200991620021_R01C01,200991620021_R03C01,200989060105_R03C01,200989060135_R07C01,200989060162_R04C01,200973410167_R04C01,200973410143_R01C01,200973410143_R06C01,...,200989060089_R02C01,200989060089_R01C01,200989060089_R04C01,200973410143_R08C01,200973410143_R07C01,200989060089_R03C01,200989060150_R04C01,200989060150_R05C01,200989060150_R06C01,200991620049_R08C01
0,cg14817997,0.8340,0.8492,0.7730,0.8853,0.7725,0.7564,0.7198,0.7759,0.7689,...,0.7882,0.8389,0.8453,0.7704,0.8529,0.7838,0.8069,0.8757,0.6613,0.7379
1,cg26928153,0.9456,0.9485,0.9622,0.9440,0.9263,0.9561,0.9130,0.9560,0.9571,...,0.9512,0.9590,0.9571,0.9547,0.9478,0.9581,0.9480,0.9424,0.9550,0.9463


In [4]:
df1 = pd.read_csv ("CpG_List_HavingCorres_GeneChr.csv")

In [5]:
df = pd.merge (df1, df, on= "CpG", how= "inner")
print(df.shape)
df.head(2)

(837962, 525)


,CpG,200973410159_R03C01,200991620021_R01C01,200991620021_R03C01,200989060105_R03C01,200989060135_R07C01,200989060162_R04C01,200973410167_R04C01,200973410143_R01C01,200973410143_R06C01,...,200989060089_R02C01,200989060089_R01C01,200989060089_R04C01,200973410143_R08C01,200973410143_R07C01,200989060089_R03C01,200989060150_R04C01,200989060150_R05C01,200989060150_R06C01,200991620049_R08C01
0,cg18478105,0.0126,0.0143,0.0155,0.0107,0.0100,0.0093,0.0134,0.0109,0.0085,...,0.0104,0.0160,0.0090,0.0106,0.0111,0.0105,0.0111,0.0116,0.0124,0.0136
1,cg09835024,0.0222,0.0295,0.0274,0.0213,0.0228,0.0442,0.0383,0.0161,0.0153,...,0.0197,0.0256,0.0227,0.0228,0.0212,0.0218,0.0247,0.0175,0.0198,0.0239


In [6]:
df1 = pd.read_csv ("CpGList_inChroXY.csv")

In [7]:
df = df.loc[df.index.difference(df1.index), ]
df.shape

(824439, 525)

In [8]:
dft = df.transpose()
dft.head(2)

,13523,13524,13525,13526,13527,13528,13529,13530,13531,13532,...,837952,837953,837954,837955,837956,837957,837958,837959,837960,837961
CpG,cg22069989,cg20517616,cg19426413,cg19305111,cg05472579,cg24694246,cg12858895,cg16943087,cg03993487,cg21490846,...,cg23659307,cg16040564,cg02455706,cg21822051,cg10615271,cg23079522,cg16818145,cg14585103,cg10633746,cg12623625
200973410159_R03C01,0.3642,0.1582,0.3494,0.015,0.0143,0.8999,0.0699,0.7524,0.0111,0.0147,...,0.8066,0.4693,0.3718,0.839,0.9298,0.8871,0.9113,0.8647,0.0615,0.6952


In [9]:
dft.rename(columns=dft.iloc[0], inplace = True)

In [10]:
dft.index.name ='CpG'
dft.reset_index(inplace=True)
dft.head(2)

,CpG,cg22069989,cg20517616,cg19426413,cg19305111,cg05472579,cg24694246,cg12858895,cg16943087,cg03993487,...,cg23659307,cg16040564,cg02455706,cg21822051,cg10615271,cg23079522,cg16818145,cg14585103,cg10633746,cg12623625
0,CpG,cg22069989,cg20517616,cg19426413,cg19305111,cg05472579,cg24694246,cg12858895,cg16943087,cg03993487,...,cg23659307,cg16040564,cg02455706,cg21822051,cg10615271,cg23079522,cg16818145,cg14585103,cg10633746,cg12623625
1,200973410159_R03C01,0.3642,0.1582,0.3494,0.015,0.0143,0.8999,0.0699,0.7524,0.0111,...,0.8066,0.4693,0.3718,0.839,0.9298,0.8871,0.9113,0.8647,0.0615,0.6952


In [11]:
dft = dft.drop ([0], axis = 0)
dft.head(2)

,CpG,cg22069989,cg20517616,cg19426413,cg19305111,cg05472579,cg24694246,cg12858895,cg16943087,cg03993487,...,cg23659307,cg16040564,cg02455706,cg21822051,cg10615271,cg23079522,cg16818145,cg14585103,cg10633746,cg12623625
1,200973410159_R03C01,0.3642,0.1582,0.3494,0.015,0.0143,0.8999,0.0699,0.7524,0.0111,...,0.8066,0.4693,0.3718,0.839,0.9298,0.8871,0.9113,0.8647,0.0615,0.6952
2,200991620021_R01C01,0.0328,0.1336,0.4513,0.017,0.0174,0.8938,0.1436,0.7146,0.0111,...,0.8088,0.4602,0.397,0.81,0.9008,0.89,0.9134,0.8209,0.1167,0.6173


In [12]:
dft.rename (columns = {"CpG" : "Sentrix"}, inplace = True)
dft.head(2)

,Sentrix,cg22069989,cg20517616,cg19426413,cg19305111,cg05472579,cg24694246,cg12858895,cg16943087,cg03993487,...,cg23659307,cg16040564,cg02455706,cg21822051,cg10615271,cg23079522,cg16818145,cg14585103,cg10633746,cg12623625
1,200973410159_R03C01,0.3642,0.1582,0.3494,0.015,0.0143,0.8999,0.0699,0.7524,0.0111,...,0.8066,0.4693,0.3718,0.839,0.9298,0.8871,0.9113,0.8647,0.0615,0.6952
2,200991620021_R01C01,0.0328,0.1336,0.4513,0.017,0.0174,0.8938,0.1436,0.7146,0.0111,...,0.8088,0.4602,0.397,0.81,0.9008,0.89,0.9134,0.8209,0.1167,0.6173


In [13]:
%cd Z:\PPMI_Data\Excels\MultiClass
df2 = pd.read_csv ("Sentrix_PATNO_CategoryHYS.csv")
df2.head(2)

Z:\PPMI_Data\Excels\MultiClass


,Sentrix,PATNO,Cat_HYS,Gender,Age,AgeCate
0,200989060089_R05C01,3115,P0,1,61.3,3
1,200989060105_R04C01,3156,P0,1,69.9,4


In [14]:
df3 = pd.merge (df2, dft, on= "Sentrix", how= "inner")
print(df3.shape)
df3.head(2)

(440, 824445)


,Sentrix,PATNO,Cat_HYS,Gender,Age,AgeCate,cg22069989,cg20517616,cg19426413,cg19305111,...,cg23659307,cg16040564,cg02455706,cg21822051,cg10615271,cg23079522,cg16818145,cg14585103,cg10633746,cg12623625
0,200989060089_R05C01,3115,P0,1,61.3,3,0.0231,0.0511,0.3999,0.0132,...,0.7339,0.8237,0.3305,0.8028,0.938,0.8486,0.9001,0.8435,0.0347,0.6676
1,200989060105_R04C01,3156,P0,1,69.9,4,0.031,0.0448,0.6598,0.0125,...,0.5244,0.8303,0.6183,0.7125,0.928,0.8704,0.9339,0.7113,0.1078,0.6428


In [15]:
df3.to_csv("Chr1_22_CpG_CatHYS_Data.csv", index = False)